In [ ]:
from HLsearch import *
from scipy.integrate import solve_ivp
from scipy import stats
import numpy as np
from sympy import symbols, var, diff, simplify, collect,solve
from sympy.utilities.lambdify import lambdify, implemented_function

from operator import add,sub,mul

import itertools

import time

def generate_data(func, time, init_values):
    sol = solve_ivp(func,[time[0],time[-1]],init_values,t_eval=time, method='RK45',rtol=1e-10,atol=1e-10)
    return sol.y.T, np.array([func(0,sol.y.T[i,:]) for i in range(sol.y.T.shape[0])],dtype=np.float64)

g = 9.81
m = 1
L = 1

def spherePend(t,y,Moment=1.0):
    theta, theta_t, phi, phi_t = y
    theta_2t, phi_2t = (L**2*m*np.sin(theta)*np.cos(theta)*phi_t**2 + L*g*m*np.sin(theta))/(L**2*m),(-2.0*L**2*m*np.sin(theta)*np.cos(theta)*phi_t*theta_t + Moment)/(L**2*m*np.sin(theta)**2)
    return theta_t,theta_2t,phi_t,phi_2t

t = np.arange(0,1.0,0.01)
y0=np.array([np.pi/2, 0,0, 0.5])
X,Xdot = generate_data(spherePend,t,y0)
pureX=X
X = X + np.random.normal(scale = 2e-8,size=X.shape)
Xdot = Xdot + np.random.normal(scale = 2e-8,size=Xdot.shape)

In [ ]:
data_description = symbols('theta theta_t phi phi_t')
print('Variables are:',data_description)
data_description_sym = data_description
data_description = list(str(descr) for descr in data_description)

expr_new0 = buildFunctionExpressions(1,round(X.shape[1]/2),[data_description[i] for i in range(round(len(data_description)/2))],use_sine=True)
expr_new1 = buildFunctionExpressions(1,round(X.shape[1]/2),[data_description[i] for i in range(round(len(data_description)/2),len(data_description))],use_sine=False)

print(expr_new0[1:])
print(expr_new1)
expr_new = expr_new0[1:]+expr_new1

expr = buildFunctionExpressions(4,len(expr_new),expr_new)

print(len(expr),' terms are: ',expr)

Theta = buildTimeSerieMatrixFromFunctions(X,expr, data_description)

Gamma = buildTimeDerivativeMatrixFromFunctions(X,Xdot,expr,data_description)

energyChange = 1.0*X[:,3]

stored_indices = ()
elements = tuple(x for x in range(len(expr)) if x not in stored_indices)
indices = itertools.combinations(elements, 3)

def countNumberOfElementsLargerThanThreshold(x,threshold = 1e-8):
    count = 0
    for i in range(len(x)):
        if abs(x[i]) > threshold:
            count = count +1
    return count

goodHamiltonian={}

start = time.time()
for count,index in enumerate(indices):
    index_tup = index + stored_indices
    xi, sumResidual = np.linalg.lstsq(Gamma[:,index_tup], energyChange,rcond=None)[:2]
    if sumResidual.size==0 or sumResidual>1e-8: continue
    if countNumberOfElementsLargerThanThreshold(xi)<=2: continue
    expr_temp = [sympify(expr[i]) for i in index_tup]
    Hamiltonian = generateExpression(xi,expr_temp,threshold=1e-8)
    Lagrangian = findLagrangianFromHamiltonian(Hamiltonian,expr_temp,data_description_sym,threshold=1e-12)
    if Lagrangian is not None and Lagrangian != '':
        goodHamiltonian[Hamiltonian] = Lagrangian
        print('Found good result at ',count,'th trial: ',index_tup)
        print('Found result after ',time.time()-start,'s')
#     print('Total Energy = ',Hamiltonian)
#     print('Found result after ',time.time()-start,'s')
print('Elapsed time: ',time.time()-start)

for H,L in goodHamiltonian.items():
    print('')
    print('Hamiltonian is ',H)
    print('Lagrangian is ',L)  

In [ ]:
def calculate(data,data_sym, function):
    data1,data2,data3,data4 = np.hsplit(data,4)
    f = lambdify(data_sym, function, 'numpy')
    return f(data1,data2,data3,data4)

pureE = calculate(pureX,data_description_sym,H)
noisyE= calculate(X,data_description_sym,H)
print(np.mean(np.abs((noisyE-pureE)/noisyE)))
print(np.std(np.abs((noisyE-pureE)/noisyE)))

In [ ]:
from sympy.physics.mechanics import *
th,ph = dynamicsymbols('theta phi')
dth,dph = dynamicsymbols('theta phi',1)
# print(dth,dx)
state = (th,dth,ph,dph)
u = dynamicsymbols('u')
L_subbed = L.subs(list(zip(data_description_sym,state)))
# mprint(L_subbed)
N = ReferenceFrame('N')
N1=N.orientnew('N1','axis',[ph,N.z])
m = (N1,u*N1.z)
LM = LagrangesMethod(L_subbed,[th,ph],forcelist=[m],frame=N)
me = LM.form_lagranges_equations()
mprint( simplify(me) )
equations = simplify(LM.rhs())
mprint(equations)